In [3]:
# Cell 1: install dependencies
!pip install scikit-image imutils scikit-learn


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 960.0 kB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.8 MB 3.3 MB/s eta 0:00:04
    --------------------------------------- 0.3/12.8 MB 3.5 MB/s eta 0:00:04
   - -------------------------------------- 0.5/12.8 MB 3.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/12.8 MB 4.3 MB/s eta 0:


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install opencv-python


   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/39.0 MB 4.8 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/39.0 MB 30.7 MB/s eta 0:00:02
   ------- -------------------------------- 7.6/39.0 MB 69.4 MB/s eta 0:00:01
   ------------- -------------------------- 13.0/39.0 MB 131.2 MB/s eta 0:00:01
   ------------------ --------------------- 18.4/39.0 MB 129.5 MB/s eta 0:00:01
   ------------------------ --------------- 23.8/39.0 MB 131.2 MB/s eta 0:00:01
   ----------------------------- ---------- 29.1/39.0 MB 131.2 MB/s eta 0:00:01
   ----------------------------------- ---- 34.3/39.0 MB 131.2 MB/s eta 0:00:01
   ---------------------------------------  39.0/39.0 MB 108.8 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 81.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Cell 2: imports & paths
import os, random
import numpy as np
import pandas as pd
from skimage import io
from skimage.transform import resize
from skimage.feature import hog
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils.object_detection import non_max_suppression

# ← Update these to match your machine:
BASE_DIR = r"C:\Users\Ali A\Desktop\School\APS360\Project\APS360-Dental-Divot-Detection\datasets\smile\combined\meta"
CSV_PATH = os.path.join(BASE_DIR, "bounding_boxes.csv")
IMAGE_DIR = os.path.join(BASE_DIR, "images_with_bboxes")

# load your annotations
ann_df = pd.read_csv(CSV_PATH)  
# must have columns: image_filename, x_min, y_min, x_max, y_max (all normalized between 0 and 1)


In [16]:
def extract_patches(image_dir, ann_df,
                    patch_size=(64,64),
                    neg_ratio=1.0,
                    max_neg_trials=50):
    pos, neg = [], []

    # detect whether CSV values look normalized (<1) or absolute (>1)
    abs_vals = ann_df[['x_min','y_min','x_max','y_max']].max().max() > 1

    for fname, group in ann_df.groupby('image_filename'):
        img_path = os.path.join(image_dir, fname)
        if not os.path.isfile(img_path):
            continue
        img = io.imread(img_path, as_gray=True)
        H, W = img.shape

        bboxes = []
        for _, r in group.iterrows():
            # skip NaNs
            if pd.isnull(r.x_min) or pd.isnull(r.y_min) \
            or pd.isnull(r.x_max) or pd.isnull(r.y_max):
                continue

            if abs_vals:
                # already pixels
                x1, y1 = int(r.x_min), int(r.y_min)
                x2, y2 = int(r.x_max), int(r.y_max)
            else:
                # normalized in [0,1]
                x1 = int(r.x_min * W)
                y1 = int(r.y_min * H)
                x2 = int(r.x_max * W)
                y2 = int(r.y_max * H)

            # sanity checks
            if x2 <= x1 or y2 <= y1:
                continue
            # clip to image bounds
            x1, y1 = max(0,x1), max(0,y1)
            x2, y2 = min(W,x2), min(H,y2)

            bboxes.append((x1, y1, x2, y2))

        # positive patches
        for x1, y1, x2, y2 in bboxes:
            patch = img[y1:y2, x1:x2]
            if patch.size == 0:
                continue
            pos.append(resize(patch, patch_size))

        # negative patches
        for _ in range(int(len(bboxes) * neg_ratio)):
            for _ in range(max_neg_trials):
                x = random.randint(0, W - patch_size[1])
                y = random.randint(0, H - patch_size[0])
                rect = (x, y, x + patch_size[1], y + patch_size[0])

                # simple IoU  
                def iou(a, b):
                    xa, ya = max(a[0], b[0]), max(a[1], b[1])
                    xb, yb = min(a[2], b[2]), min(a[3], b[3])
                    inter = max(0, xb - xa) * max(0, yb - ya)
                    A = (a[2] - a[0]) * (a[3] - a[1])
                    B = (b[2] - b[0]) * (b[3] - b[1])
                    return inter / (A + B - inter) if inter > 0 else 0

                if all(iou(rect, bb) < 0.1 for bb in bboxes):
                    neg_patch = img[y:y + patch_size[0], x:x + patch_size[1]]
                    if neg_patch.size == 0:
                        continue
                    neg.append(resize(neg_patch, patch_size))
                    break

    return pos, neg

pos_patches, neg_patches = extract_patches(IMAGE_DIR, ann_df)
print(f"Got {len(pos_patches)} positive patches and {len(neg_patches)} negative patches.")


Got 5355 positive patches and 5355 negative patches.


In [17]:
import pandas as pd

print("Total rows:", len(ann_df))
print("Rows with any NaN in bbox coords:", 
      ann_df[['x_min','y_min','x_max','y_max']].isnull().any(axis=1).sum())

print("Unique filenames:", ann_df['image_filename'].nunique())
print("Sample filenames:", ann_df['image_filename'].unique()[:10])


Total rows: 5355
Rows with any NaN in bbox coords: 0
Unique filenames: 973
Sample filenames: ['dc1000_195.png' 'dc1000_181.png'
 'dental_radiography_0078_jpg.rf.3df5e76aaf3853ffeb55283ed9666c1e.jpg'
 'dc1000_630.png' 'dc1000_156.png' 'dc1000_142.png' 'dc1000_624.png'
 'dental_radiography_0054_jpg.rf.b81c1de4282e2881bc92f9d5b6ca106f.jpg'
 'opg_xray_5.jpg' 'dc1000_618.png']


In [18]:
# Cell 4: HOG feature extraction & dataset prep
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split

def compute_hog_features(patches):
    return np.array([
        hog(p,
            orientations=9,
            pixels_per_cell=(8,8),
            cells_per_block=(2,2),
            block_norm='L2-Hys')
        for p in patches
    ])

X_pos = compute_hog_features(pos_patches)
X_neg = compute_hog_features(neg_patches)
y_pos = np.ones(len(X_pos))
y_neg = np.zeros(len(X_neg))

# combine and split
X = np.vstack([X_pos, X_neg])
y = np.concatenate([y_pos, y_neg])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train set:", X_train.shape, y_train.sum(), "positives")
print("Test set: ", X_test.shape, y_test.sum(), "positives")


Train set: (8568, 1764) 4284.0 positives
Test set:  (2142, 1764) 1071.0 positives


In [19]:
# Cell 5: train & evaluate the linear SVM classifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

clf = make_pipeline(
    StandardScaler(),
    LinearSVC(max_iter=5000, random_state=42)
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99      1071
         1.0       0.99      1.00      0.99      1071

    accuracy                           0.99      2142
   macro avg       0.99      0.99      0.99      2142
weighted avg       0.99      0.99      0.99      2142



In [20]:
# Cell 6: sliding-window detector + NMS demo on a real image
from imutils.object_detection import non_max_suppression
from skimage.transform import resize

def sliding_window(img, step, ws):
    for y in range(0, img.shape[0] - ws[1] + 1, step):
        for x in range(0, img.shape[1] - ws[0] + 1, step):
            yield x, y, img[y:y+ws[1], x:x+ws[0]]

def detect(img, clf, ws=(64,64), step=16, thresh=0.5):
    rects, scores = [], []
    for x, y, win in sliding_window(img, step, ws):
        feat = hog(resize(win, ws),
                   orientations=9,
                   pixels_per_cell=(8,8),
                   cells_per_block=(2,2),
                   block_norm='L2-Hys')
        score = clf.decision_function([feat])[0]
        if score > thresh:
            rects.append((x, y, x+ws[0], y+ws[1]))
            scores.append(score)
    picks = non_max_suppression(np.array(rects), scores, overlapThresh=0.3)
    return picks

# choose a sample image from your annotations
test_fname = ann_df['image_filename'].iloc[0]
test_path  = os.path.join(IMAGE_DIR, test_fname)
test_img   = io.imread(test_path, as_gray=True)

boxes = detect(test_img, clf)
print(f"Detections for {test_fname}: {boxes}")


Detections for dc1000_195.png: [[  80   16  144   80]
 [ 928    0  992   64]
 [ 160   16  224   80]
 [ 240    0  304   64]
 [2816   16 2880   80]
 [   0    0   64   64]
 [ 736  560  800  624]
 [1472  112 1536  176]
 [2768   16 2832   80]]


In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# 5b) overall accuracy & error rate
acc = accuracy_score(y_test, y_pred)
err = 1 - acc
print(f"Overall accuracy: {acc:.4f}")
print(f"Error rate:       {err:.4f}")

# 5c) confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix (rows=true, cols=predicted):")
print(cm)

# Optionally, cross-validation to estimate variance
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print(f"5-fold CV accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")


Overall accuracy: 0.9930
Error rate:       0.0070
Confusion matrix (rows=true, cols=predicted):
[[1057   14]
 [   1 1070]]
5-fold CV accuracy: 0.9700 ± 0.0551
